## Import Required Libraries

In [1]:
import os
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import torch
from datetime import datetime
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm
from ultralytics import YOLO
from roboflow import Roboflow
from SoccerNet.Downloader import SoccerNetDownloader

c:\Users\Ali Riza Ercan\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


## Set up directories

In [2]:
BASE_DIR = os.path.abspath('')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Connect to Roboflow and Download Dataset

In [ ]:
def download_dataset():
    rf = Roboflow(api_key="your-api-key")
    project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
    version = project.version(8)
    dataset = version.download("yolov8")
    print(f"Dataset downloaded to: {dataset.location}")
    return dataset

## Download SoccerNet Dataset (corrected version)

In [4]:
def download_soccernet_data():
    try:
        # Create SoccerNet directory if it doesn't exist
        soccernet_dir = os.path.join(DATA_DIR, 'soccernet')
        os.makedirs(soccernet_dir, exist_ok=True)
        
        # Initialize downloader with correct parameters
        downloader = SoccerNetDownloader()
        
        # Download specific features
        downloader.downloadGames(
            files=["1_ResNET_TF2.npy", "2_ResNET_TF2.npy"],  # Updated feature names
            split=["train", "valid", "test"],
            destination=soccernet_dir
        )
        print("SoccerNet data downloaded successfully")
    except Exception as e:
        print(f"Error downloading SoccerNet data: {e}")

## Train YOLOv8 Model

In [5]:
def train_yolov8_model(dataset):
    """Train YOLOv8 model on soccer players and ball dataset"""
    model = YOLO('yolov8n.pt')  # Load pretrained model
    
    results = model.train(
        data=os.path.join(dataset.location, 'data.yaml'),
        epochs=50,
        imgsz=640,
        batch=16,
        name='soccer_analysis',
        device=0 if torch.cuda.is_available() else 'cpu'
    )
    
    model_path = os.path.join(MODELS_DIR, 'soccer_analysis_yolov8.pt')
    model.save(model_path)
    print(f"Model trained and saved to: {model_path}")
    return model_path

## Define Formation Detection Function

In [6]:
def detect_formation(player_positions, pitch_dimensions=(105, 68)):
    """
    Detect team formation based on player positions
    
    Args:
        player_positions: List of (x, y) player coordinates
        pitch_dimensions: Tuple of (length, width) in meters
        
    Returns:
        formation: String representing formation (e.g., "4-3-3", "4-4-2")
    """
    if len(player_positions) < 10:  # Excluding goalkeeper
        return "Unknown"
    
    # Normalize coordinates to pitch dimensions
    normalized_positions = []
    for x, y in player_positions:
        norm_x = x / pitch_dimensions[0]
        norm_y = y / pitch_dimensions[1]
        normalized_positions.append((norm_x, norm_y))
    
    # Sort players by y-position (distance from own goal)
    normalized_positions.sort(key=lambda pos: pos[1])
    
    # Remove goalkeeper (assuming keeper is closest to own goal)
    field_players = normalized_positions[1:]
    
    # Use KMeans to cluster players into different lines
    positions = np.array(field_players)
    kmeans = KMeans(n_clusters=3, random_state=0).fit(positions)
    labels = kmeans.labels_
    
    # Count players in each line
    line_counts = [sum(labels == i) for i in range(3)]
    
    # Determine formation based on line counts
    formation = f"{line_counts[0]}-{line_counts[1]}-{line_counts[2]}"
    
    # Handle common variations
    if formation == "4-3-3":
        return "4-3-3"
    elif formation == "4-4-2":
        return "4-4-2"
    elif formation == "3-5-2":
        return "3-5-2"
    elif formation == "5-3-2":
        return "5-3-2"
    elif formation == "3-4-3":
        return "3-4-3"
    else:
        return formation

## Ball Possession Analysis

In [7]:
def calculate_possession(ball_detections, home_team_detections, away_team_detections):
    """
    Calculate possession percentages based on proximity of ball to players
    
    Args:
        ball_detections: List of ball coordinates per frame
        home_team_detections: List of home player coordinates per frame
        away_team_detections: List of away player coordinates per frame
        
    Returns:
        home_possession: Percentage of frames where home team has possession
        away_possession: Percentage of frames where away team has possession
    """
    home_possession_frames = 0
    away_possession_frames = 0
    no_possession_frames = 0
    
    for i, ball_pos in enumerate(ball_detections):
        if not ball_pos:
            no_possession_frames += 1
            continue
            
        home_players = home_team_detections[i]
        away_players = away_team_detections[i]
        
        if not home_players and not away_players:
            no_possession_frames += 1
            continue
            
        # Calculate minimum distance to players
        min_distance_home = min([np.linalg.norm(np.array(ball_pos) - np.array(player)) 
                                for player in home_players]) if home_players else float('inf')
        min_distance_away = min([np.linalg.norm(np.array(ball_pos) - np.array(player)) 
                                for player in away_players]) if away_players else float('inf')
        
        # Assign possession to the team with the closest player
        if min_distance_home < min_distance_away:
            home_possession_frames += 1
        else:
            away_possession_frames += 1
    
    total_frames = home_possession_frames + away_possession_frames + no_possession_frames
    
    if total_frames == 0:
        return 0, 0
        
    home_possession = (home_possession_frames / total_frames) * 100
    away_possession = (away_possession_frames / total_frames) * 100
    
    return home_possession, away_possession

## Create Player Heatmaps

In [8]:
def generate_heatmaps(team_detections, pitch_dimensions=(105, 68), resolution=(100, 65)):
    """
    Generate heatmap data for player positions
    
    Args:
        team_detections: List of player positions for all frames
        pitch_dimensions: Actual pitch size in meters
        resolution: Resolution of the heatmap grid
        
    Returns:
        heatmap: 2D numpy array representing player position density
    """
    heatmap = np.zeros(resolution)
    
    for frame_detections in team_detections:
        for player_pos in frame_detections:
            # Normalize to heatmap grid
            grid_x = int((player_pos[0] / pitch_dimensions[0]) * (resolution[0] - 1))
            grid_y = int((player_pos[1] / pitch_dimensions[1]) * (resolution[1] - 1))
            
            # Ensure within bounds
            grid_x = max(0, min(grid_x, resolution[0] - 1))
            grid_y = max(0, min(grid_y, resolution[1] - 1))
            
            heatmap[grid_y, grid_x] += 1
    
    # Normalize heatmap
    if np.max(heatmap) > 0:
        heatmap = heatmap / np.max(heatmap)
    
    return heatmap.tolist()  # Convert to list for JSON serialization

## Main Video Analysis Function

In [9]:
def analyze_soccer_match(video_path, model_path, home_team, away_team, match_date):
    """
    Perform comprehensive analysis of a soccer match video
    
    Args:
        video_path: Path to the match video file
        model_path: Path to the trained YOLOv8 model
        home_team: Name of the home team
        away_team: Name of the away team
        match_date: Date of the match (YYYY-MM-DD)
        
    Returns:
        analysis_data: Dictionary containing all analysis results
    """
    # Load the model
    model = YOLO(model_path)
    
    # Open video file
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_seconds = total_frames / fps
    
    # Define team colors for visualization
    home_color = (0, 0, 255)  # Red
    away_color = (255, 0, 0)  # Blue
    ball_color = (0, 255, 0)  # Green
    
    # Initialize tracking data
    home_team_detections = []
    away_team_detections = []
    ball_detections = []
    
    # Process every 5th frame to reduce computation
    frame_interval = 5
    processed_frames = 0
    
    # Time-based statistics
    possession_timeline = []
    formation_changes = []
    current_home_formation = "Unknown"
    current_away_formation = "Unknown"
    home_formation_changes = 0
    away_formation_changes = 0
    
    # Shot statistics (would need more sophisticated detection in practice)
    home_shots = 0
    away_shots = 0
    home_shots_on_target = 0
    away_shots_on_target = 0
    
    # Pass accuracy (placeholder - would need more sophisticated tracking)
    home_pass_accuracy = 0.0
    away_pass_accuracy = 0.0
    
    print(f"Analyzing video with {total_frames} frames ({duration_seconds:.2f} seconds)")
    
    for frame_idx in tqdm(range(0, total_frames, frame_interval), desc="Processing frames"):
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = video.read()
        
        if not ret:
            break
            
        # Perform detection
        results = model(frame)
        
        # Extract detections for this frame
        frame_home_detections = []
        frame_away_detections = []
        frame_ball_detection = None
        
        for result in results:
            boxes = result.boxes.cpu().numpy()
            for i, box in enumerate(boxes):
                cls = int(box.cls[0])
                conf = box.conf[0]
                
                if conf < 0.4:  # Confidence threshold
                    continue
                    
                x1, y1, x2, y2 = box.xyxy[0].astype(int)
                center_x = (x1 + x2) / 2
                center_y = (y1 + y2) / 2
                
                # Class mapping (adjust based on your model's classes)
                # Assuming: 0 = home player, 1 = away player, 2 = ball
                if cls == 0:  # Home player
                    frame_home_detections.append((center_x, center_y))
                elif cls == 1:  # Away player
                    frame_away_detections.append((center_x, center_y))
                elif cls == 2:  # Ball
                    frame_ball_detection = (center_x, center_y)
        
        # Store detections
        home_team_detections.append(frame_home_detections)
        away_team_detections.append(frame_away_detections)
        ball_detections.append(frame_ball_detection)
        
        # Update formation every 30 seconds (30*fps/frame_interval frames)
        formation_check_interval = int(30 * fps / frame_interval)
        if processed_frames % formation_check_interval == 0:
            # Detect formations
            home_formation = detect_formation(frame_home_detections)
            away_formation = detect_formation(frame_away_detections)
            
            # Check for formation changes
            if home_formation != current_home_formation and current_home_formation != "Unknown":
                home_formation_changes += 1
                formation_changes.append({
                    "team": "home",
                    "time": processed_frames * frame_interval / fps,
                    "old_formation": current_home_formation,
                    "new_formation": home_formation
                })
            
            if away_formation != current_away_formation and current_away_formation != "Unknown":
                away_formation_changes += 1
                formation_changes.append({
                    "team": "away",
                    "time": processed_frames * frame_interval / fps,
                    "old_formation": current_away_formation,
                    "new_formation": away_formation
                })
            
            current_home_formation = home_formation
            current_away_formation = away_formation
        
        # Update possession timeline every 10 seconds
        possession_check_interval = int(10 * fps / frame_interval)
        if processed_frames % possession_check_interval == 0 and frame_ball_detection:
            # Determine possession for this segment
            possession = "none"
            if frame_home_detections and frame_away_detections:
                home_distances = [np.linalg.norm(np.array(frame_ball_detection) - np.array(player)) 
                                  for player in frame_home_detections]
                away_distances = [np.linalg.norm(np.array(frame_ball_detection) - np.array(player)) 
                                  for player in frame_away_detections]
                
                min_home_distance = min(home_distances) if home_distances else float('inf')
                min_away_distance = min(away_distances) if away_distances else float('inf')
                
                possession = "home" if min_home_distance < min_away_distance else "away"
            
            possession_timeline.append({
                "time": processed_frames * frame_interval / fps,
                "possession": possession
            })
        
        processed_frames += 1
    
    # Calculate overall possession percentages
    home_possession, away_possession = calculate_possession(
        ball_detections, home_team_detections, away_team_detections
    )
    
    # Generate heatmaps
    home_heatmap = generate_heatmaps(home_team_detections)
    away_heatmap = generate_heatmaps(away_team_detections)
    
    # Finalize formations
    if current_home_formation == "Unknown" and home_team_detections:
        # Try to detect formation from aggregated positions
        all_home_positions = [pos for frame in home_team_detections for pos in frame]
        current_home_formation = detect_formation(all_home_positions)
    
    if current_away_formation == "Unknown" and away_team_detections:
        # Try to detect formation from aggregated positions
        all_away_positions = [pos for frame in away_team_detections for pos in frame]
        current_away_formation = detect_formation(all_away_positions)
    
    # Clean up
    video.release()
    
    # Create analysis data structure
    analysis_data = {
        "match_date": match_date,
        "video_path": video_path,
        "home_team": home_team,
        "away_team": away_team,
        "home_possession_percentage": home_possession,
        "away_possession_percentage": away_possession,
        "home_pass_accuracy": home_pass_accuracy,
        "away_pass_accuracy": away_pass_accuracy,
        "home_shots": home_shots,
        "away_shots": away_shots,
        "home_shots_on_target": home_shots_on_target,
        "away_shots_on_target": away_shots_on_target,
        "home_formation": current_home_formation,
        "away_formation": current_away_formation,
        "home_formation_changes": home_formation_changes,
        "away_formation_changes": away_formation_changes,
        "home_heatmap": home_heatmap,
        "away_heatmap": away_heatmap,
        "possession_timeline": possession_timeline,
        "formation_changes": formation_changes
    }
    
    return analysis_data

## User Input and Full Pipeline

In [10]:
def process_match_video(video_path, home_team, away_team, match_date_str):
    """
    Process a match video and save analysis as JSON file
    
    Args:
        video_path: Path to the match video file
        home_team: Name of the home team
        away_team: Name of the away team
        match_date_str: Date of the match as string (YYYY-MM-DD)
        
    Returns:
        output_file: Path to the saved analysis JSON file
    """
    # Convert date string to date object
    match_date = datetime.strptime(match_date_str, "%Y-%m-%d").date()
    
    # Get or train model
    model_path = os.path.join(MODELS_DIR, 'soccer_analysis_yolov8.pt')
    if not os.path.exists(model_path):
        # First download the dataset
        dataset = download_dataset()
        # Then train the model with the dataset
        model_path = train_yolov8_model(dataset)
    
    # Analyze the video
    analysis_data = analyze_soccer_match(
        video_path, model_path, home_team, away_team, match_date
    )
    
    # Create a clean filename
    clean_home = home_team.replace(" ", "_")
    clean_away = away_team.replace(" ", "_")
    filename = f"{clean_home}_vs_{clean_away}_{match_date_str}.json"
    output_file = os.path.join(ANALYSIS_DIR, filename)
    
    # Save analysis to file
    with open(output_file, 'w') as f:
        json.dump(analysis_data, f, indent=4)
    
    print(f"Match analysis saved to: {output_file}")
    return output_file

## Load and Analyze Previously Saved Match Analysis

In [11]:
def load_match_analysis(analysis_file):
    """Load a previously saved match analysis"""
    with open(analysis_file, 'r') as f:
        analysis_data = json.load(f)
    return analysis_data

## Visualization Functions for Analysis Results

In [12]:
def visualize_heatmap(heatmap_data, title):
    """Visualize player position heatmap"""
    plt.figure(figsize=(12, 8))
    plt.imshow(heatmap_data, cmap='hot', interpolation='nearest')
    plt.colorbar(label='Normalized presence')
    plt.title(title)
    plt.xlabel('Field length')
    plt.ylabel('Field width')
    plt.show()

def visualize_possession_timeline(possession_timeline):
    """Visualize possession changes over time"""
    times = [entry['time'] for entry in possession_timeline]
    possession_values = []
    
    for entry in possession_timeline:
        if entry['possession'] == 'home':
            possession_values.append(1)
        elif entry['possession'] == 'away':
            possession_values.append(-1)
        else:
            possession_values.append(0)
    
    plt.figure(figsize=(15, 5))
    plt.plot(times, possession_values, '-o')
    plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    plt.fill_between(times, possession_values, 0, where=(np.array(possession_values) > 0), 
                     color='red', alpha=0.3, label='Home Possession')
    plt.fill_between(times, possession_values, 0, where=(np.array(possession_values) < 0), 
                     color='blue', alpha=0.3, label='Away Possession')
    plt.yticks([-1, 0, 1], ['Away', 'None', 'Home'])
    plt.xlabel('Time (seconds)')
    plt.ylabel('Possession')
    plt.title('Possession Timeline')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def visualize_formation_changes(formation_changes, match_duration):
    """Visualize formation changes during the match"""
    home_changes = [fc for fc in formation_changes if fc['team'] == 'home']
    away_changes = [fc for fc in formation_changes if fc['team'] == 'away']
    
    plt.figure(figsize=(15, 6))
    
    # Plot home team formation changes
    for i, change in enumerate(home_changes):
        plt.plot([change['time'], change['time']], [0.5, 1], 'r-', linewidth=2)
        plt.text(change['time'], 1.05, f"{change['old_formation']} → {change['new_formation']}", 
                ha='center', va='bottom', rotation=90, color='red')
    
    # Plot away team formation changes
    for i, change in enumerate(away_changes):
        plt.plot([change['time'], change['time']], [0.5, 0], 'b-', linewidth=2)
        plt.text(change['time'], -0.05, f"{change['old_formation']} → {change['new_formation']}", 
                ha='center', va='top', rotation=90, color='blue')
    
    plt.axhline(y=0.5, color='gray', linestyle='-', alpha=0.5)
    plt.fill_between([0, match_duration], [0.5, 0.5], [1, 1], color='red', alpha=0.1, label='Home Team')
    plt.fill_between([0, match_duration], [0.5, 0.5], [0, 0], color='blue', alpha=0.1, label='Away Team')
    
    plt.xlim(0, match_duration)
    plt.ylim(-0.2, 1.2)
    plt.yticks([0.25, 0.75], ['Away Team', 'Home Team'])
    plt.xlabel('Time (seconds)')
    plt.title('Formation Changes During Match')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

def generate_match_report(analysis_data):
    """Generate a markdown report from the analysis data"""
    home_team = analysis_data['home_team']
    away_team = analysis_data['away_team']
    match_date = analysis_data['match_date']
    
    # Calculate match statistics
    home_poss = analysis_data['home_possession_percentage']
    away_poss = analysis_data['away_possession_percentage']
    
    report = f"""# Match Analysis: {home_team} vs {away_team}
Date: {match_date}

## Match Summary
- **Home Formation**: {analysis_data['home_formation']}
- **Away Formation**: {analysis_data['away_formation']}
- **Home Possession**: {home_poss:.1f}%
- **Away Possession**: {away_poss:.1f}%
- **Home Shots (On Target)**: {analysis_data['home_shots']} ({analysis_data['home_shots_on_target']})
- **Away Shots (On Target)**: {analysis_data['away_shots']} ({analysis_data['away_shots_on_target']})

## Formation Changes
- **Home Team**: {analysis_data['home_formation_changes']} changes
- **Away Team**: {analysis_data['away_formation_changes']} changes

## Tactical Analysis
The match analysis reveals that {home_team} primarily used a {analysis_data['home_formation']} formation, 
while {away_team} countered with a {analysis_data['away_formation']} setup.

Possession was {'evenly split' if abs(home_poss - away_poss) < 10 else f"dominated by {home_team if home_poss > away_poss else away_team}"}, 
with {home_team if home_poss > away_poss else away_team} controlling {max(home_poss, away_poss):.1f}% of the play.

"""
    
    # Add formation changes if any
    if analysis_data['formation_changes']:
        report += "\n### Key Formation Changes\n"
        for change in analysis_data['formation_changes']:
            team_name = home_team if change['team'] == 'home' else away_team
            time_mins = int(change['time'] / 60)
            time_secs = int(change['time'] % 60)
            report += f"- {team_name} changed from {change['old_formation']} to {change['new_formation']} at {time_mins}:{time_secs:02d}\n"
    
    # Save the report to a file
    clean_home = home_team.replace(" ", "_")
    clean_away = away_team.replace(" ", "_")
    report_filename = f"{clean_home}_vs_{clean_away}_{match_date}_report.md"
    report_path = os.path.join(OUTPUT_DIR, report_filename)
    
    with open(report_path, 'w') as f:
        f.write(report)
    
    print(f"Match report saved to: {report_path}")
    return report

## Example Usage

In [13]:
def main():
    # This would typically be gathered from user input
    video_path = "konyaspor_example.mp4"
    home_team = "Konyaspor"
    away_team = "Iskenderunspor"
    match_date = "2025-04-01"
    
    # Process the match video
    analysis_file = process_match_video(video_path, home_team, away_team, match_date)
    
    # Load the analysis for visualization
    analysis_data = load_match_analysis(analysis_file)
    
    # Visualize results
    visualize_heatmap(analysis_data['home_heatmap'], f"{home_team} Player Positions")
    visualize_heatmap(analysis_data['away_heatmap'], f"{away_team} Player Positions")
    visualize_possession_timeline(analysis_data['possession_timeline'])
    
    # Get video duration
    try:
        video = cv2.VideoCapture(video_path)
        fps = video.get(cv2.CAP_PROP_FPS)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        duration_seconds = total_frames / fps
        video.release()
    except:
        # Fallback duration if video can't be opened
        duration_seconds = 5400  # 90 minutes in seconds
    
    visualize_formation_changes(analysis_data['formation_changes'], duration_seconds)
    
    # Generate a markdown report
    generate_match_report(analysis_data)
    
    print("Analysis completed successfully!")

if __name__ == "__main__":
    main()

loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: c:\Users\Ali Riza Ercan\Documents\GitHub\IQAnadoluScout (IQAS)\IQAnadoluScout--IQAS-\notebooks\football-players-detection-8
Ultralytics 8.3.107  Python-3.9.0 torch-2.6.0+cpu CPU (Intel Core(TM) i5-10300H 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=c:\Users\Ali Riza Ercan\Documents\GitHub\IQAnadoluScout (IQAS)\IQAnadoluScout--IQAS-\notebooks\football-players-detection-8\data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=soccer_analysis2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=3

train: Scanning C:\Users\Ali Riza Ercan\Documents\GitHub\IQAnadoluScout (IQAS)\IQAnadoluScout--IQAS-\notebooks\football-players-detection-8\train\labels.cache... 204 images, 0 backgrounds, 0 corrupt: 100%|██████████| 204/204 [00:00<?, ?it/s]
val: Scanning C:\Users\Ali Riza Ercan\Documents\GitHub\IQAnadoluScout (IQAS)\IQAnadoluScout--IQAS-\notebooks\football-players-detection-8\valid\labels.cache... 38 images, 0 backgrounds, 0 corrupt: 100%|██████████| 38/38 [00:00<?, ?it/s]


Plotting labels to C:\Users\Ali Riza Ercan\runs\detect\soccer_analysis2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\Ali Riza Ercan\runs\detect\soccer_analysis2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.888      3.939     0.9375        458        640: 100%|██████████| 13/13 [02:26<00:00, 11.29s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.12s/it]

                   all         38        905     0.0108     0.0444    0.00637    0.00247


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.21s/it]


                   all         38        905       0.85      0.565       0.63      0.397
                  ball         35         35          1          0     0.0269    0.00269
            goalkeeper         27         27      0.869       0.74      0.889      0.566
                player         38        754      0.886       0.93      0.962      0.645
               referee         38         89      0.645      0.591      0.641      0.374
Speed: 4.2ms preprocess, 78.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to C:\Users\Ali Riza Ercan\runs\detect\soccer_analysis2
Model trained and saved to: c:\Users\Ali Riza Ercan\Documents\GitHub\IQAnadoluScout (IQAS)\IQAnadoluScout--IQAS-\notebooks\models\soccer_analysis_yolov8.pt
Analyzing video with 13515 frames (540.60 seconds)


Processing frames:   0%|          | 0/2703 [00:00<?, ?it/s]


0: 384x640 1 player, 153.6ms
Speed: 7.2ms preprocess, 153.6ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 player, 102.2ms
Speed: 3.0ms preprocess, 102.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 player, 111.1ms
Speed: 3.4ms preprocess, 111.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 player, 90.9ms
Speed: 3.3ms preprocess, 90.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 player, 125.0ms
Speed: 5.5ms preprocess, 125.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 player, 103.3ms
Speed: 3.4ms preprocess, 103.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 player, 87.6ms
Speed: 2.8ms preprocess, 87.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 96.2ms
Speed: 5.9ms preprocess, 96.2ms inference, 0.8ms postprocess per image a

IndexError: index 99 is out of bounds for axis 1 with size 65